## **A simple Q&A application over a PDF data source**

### **Installing Required Libraries**

In [ ]:
%pip install langchain langchain-google-genai langchain-community faiss-cpu PyPDF2

In [ ]:
%pip install --upgrade pip

### **API Key Configuration**

In [ ]:
import os

os.environ["GOOGLE_API_KEY"] = "chave-aqui"

### **Basic RAG**

In [41]:
# Libriries
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
import PyPDF2

#### **Sobre o documento utilizado**

Boletim informativo nº 88 da Câmara de Comercialização de Energia Elétrica (CCEE), referente ao mês de agosto de 2025. O objetivo principal do boletim é detalhar a memória de cálculo utilizada como subsídio para a ANEEL (Agência Nacional de Energia Elétrica) no processo de acionamento das bandeiras tarifárias.

In [ ]:
# Define uma variável 'path' com o caminho do arquivo PDF a ser lido.
path = "/content/bandeira_tarifaria.pdf"

# Inicia uma variável de texto vazia para armazenar o conteúdo do PDF
document = ""

# Abre o arquivo PDF no caminho especificado
with open(path, 'rb') as arquivo_pdf:

  # Carrega o arquivo na ferramenta que sabe como ler o conteúdo de PDFs
  leitor_pdf = PyPDF2.PdfReader(arquivo_pdf)

  # Cria um laço para "folhear" cada página do arquivo, uma por uma
  for pagina in leitor_pdf.pages:

    # Pega todo o texto da página atual e o adiciona à nossa variável
    document += pagina.extract_text() + "\n"

In [54]:
document

' \n \n \n \nwww.ccee.org.br Nº 88 - agosto/2025 0800 881 2233 \n \n \n \n \n \n \nDesde novembro de 2017, a metodologia de acionamento das bandeiras tarifárias se \nalterou, com a finalidade de melhorar a arrecadação de recursos para fazer frente a \nimportantes obrigações financeiras de curto prazo que recaem sobre o fluxo de caixa \ndas Distribuidoras, vinculados aos custos variáveis (custos de geração por fonte \ntermelétrica e da exposição aos preços de liquidação no mercado de curto prazo) \ndecorrentes do resultado da operação do Sistema Interligado Nacional – SIN1. \nA sistemática de acionamento da bandeira tarifária aplicada até o mês de abril de 2018 \nseguia o descrito nas Notas Técnicas nº 133/2017-SRG-SEM-SGT/ANEEL e nº 136/2017-\nSRG-SEM-SGT/ANEEL, que suportavam a Audiência Pública nº 61/2017, da ANEEL, que \ntinha como objetivo obter subsídios para a revisão da metodologia das Bandeiras \nTarifárias. O voto do diretor relator do processo na ANEEL, que decidiu pela abert

In [55]:
# Cria e configura a nossa "tesoura inteligente" para cortar o texto
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,   # Define que cada pedaço de texto terá, no máximo, 1000 caracteres
    length_function=len,  # Informa qual a função para medir o tamanho do texto
    is_separator_regex=False,  # Uma configuração técnica para os separadores abaixo
    # Define a prioridade de corte: a tesoura tentará cortar primeiro
    # onde houver um parágrafo em branco, para manter as ideias juntas
    separators=["\n\n"],
)

# Pega o nosso documento grande e usa a "tesoura" para cortá-lo
# em uma lista de vários pedaços de texto menores
texts = text_splitter.create_documents([document])

# Prepara o "tradutor de significados" do Google. É ele que vai converter
# os pedaços de texto em números que o computador entende
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

# Cria a "biblioteca" ou "índice" do nosso documento. Ele pega os pedaços
# de texto, usa o "tradutor" para criar uma versão numérica de cada um,
# e organiza tudo para permitir uma busca de informações muito rápida
vector_store = FAISS.from_documents(texts, embedding=embeddings)

In [56]:
print(texts)

[Document(metadata={}, page_content=' \n \n \n \nwww.ccee.org.br Nº 88 - agosto/2025 0800 881 2233 \n \n \n \n \n \n \nDesde novembro de 2017, a metodologia de acionamento das bandeiras tarifárias se \nalterou, com a finalidade de melhorar a arrecadação de recursos para fazer frente a \nimportantes obrigações financeiras de curto prazo que recaem sobre o fluxo de caixa \ndas Distribuidoras, vinculados aos custos variáveis (custos de geração por fonte \ntermelétrica e da exposição aos preços de liquidação no mercado de curto prazo) \ndecorrentes do resultado da operação do Sistema Interligado Nacional – SIN1. \nA sistemática de acionamento da bandeira tarifária aplicada até o mês de abril de 2018 \nseguia o descrito nas Notas Técnicas nº 133/2017-SRG-SEM-SGT/ANEEL e nº 136/2017-\nSRG-SEM-SGT/ANEEL, que suportavam a Audiência Pública nº 61/2017, da ANEEL, que \ntinha como objetivo obter subsídios para a revisão da metodologia das Bandeiras \nTarifárias. O voto do diretor relator do proce

In [57]:
# É um texto que diz exatamente qual é o trabalho da IA
template="""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.

Question: {question}

Context: {context}

Answer (brazilian portuguese):"""

# Esta linha pega o roteiro acima e o transforma em um "molde" formal,
# que o sistema usará para encaixar a pergunta e os textos de forma automática
prompt = ChatPromptTemplate.from_template(template)

# Aqui, nós "ligamos" e configuramos o cérebro da operação (o modelo de IA do Google).
llm = ChatGoogleGenerativeAI(
    # Escolhemos a versão do cérebro a ser usada: o "gemini-1.5-flash".
    model="gemini-1.5-flash",
    # Com 'temperatura' 0, a IA será 100% objetiva e fiel ao texto, sem usar a criatividade.
    temperature=0,
    # Define o tamanho máximo da resposta (aqui, sem um limite específico).
    max_tokens=None,
    # Define o tempo máximo de espera por uma resposta.
    timeout=None,
    # Se a conexão falhar, ele tentará se reconectar mais 2 vezes.
    max_retries=2,
)

In [58]:
# Faz uma busca na "biblioteca" que criamos para encontrar os 2 trechos
# de texto mais parecidos com a sua pergunta

# question = "Qual é a cor da bandeira tarifária para agosto de 2025?"
# question = "Qual foi o valor do PLD gatilho considerado para agosto de 2025?"
question = "Qual o objetivo deste boletim?"

retrieved_docs = vector_store.similarity_search(question, k=2)

In [59]:
# Junta os trechos de texto encontrados em um único bloco de texto,
# separado por um espaço, para facilitar a leitura pela IA
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

# Preenche o "molde de instruções" que definimos antes. Ele insere a sua
# pergunta e os textos encontrados nos seus devidos lugares
messages = prompt.invoke({"question": question, "context": docs_content})

# Envia o prompt final (com as instruções, a pergunta e o contexto)
# para o "cérebro" da IA, que então gera a resposta
response = llm.invoke(messages)

In [60]:
response.content

'O objetivo deste boletim é detalhar a memória de cálculo considerada pela ANEEL como subsídio para o acionamento das bandeiras tarifárias.'